#### 지도(folium) 활용 방안 - 서울시 자치구별 태양광 발전

목적: 자치구별 태양광 발전소 분포와 설비용량 현황을 파악하고 지도 시각화를 통해 지역별 차이를 직관적으로 비교

분석 과정
 - 도로명주소에서 자치구 정보 추출
 - 자치구 단위로 발전소 개수 및 총 설비용량 집계
 - 발전소 개수 기준 최다/최소 자치구 식별
 - folium 지도를 활용한 자치구별 마커 시각화 및 강조 표시

데이터
 - 서울 열린 데이터 광장 - 서울시 태양광발전사업 정보.csv

In [2]:
import pandas as pd

df = pd.read_csv("../data/서울시_태양광발전사업_정보.csv", encoding='cp949')

# 컬럼 확인
print("컬럼 목록:")
print(df.columns.tolist())

print("\n데이터 미리보기:")
print(df.head())

컬럼 목록:
['태양광발전시설명', '소재지도로명주소', '가동상태구분명', '설비용량(kw)', '공급전압(V)', '주파수(Hz)', '설치연도', '데이터기준일자']

데이터 미리보기:
       태양광발전시설명                          소재지도로명주소 가동상태구분명  설비용량(kw)  공급전압(V)  \
0     제일교회햇빛발전소                 서울시 중구 오장동 101-21     가동중      19.2    380.0   
1  원불교 요안 햇빛발전소  서울시 강서구 가로공원로84길 59(화곡1동 352-46)     가동중       3.0      NaN   
2    수도전기공업고등학교          서울시 강남구 개포로 410(개포동 155)     가동중      90.0      NaN   
3    ?일조각 파워하우스      서울시 종로구 신문로2가 1-335, 일조각 출판사     가동중       3.0      NaN   
4   재단법인 아산복지재단     서울시 송파구 올림픽로43길 88(풍납동 388-1)     가동중     147.0      NaN   

   주파수(Hz)    설치연도     데이터기준일자  
0       60  2018.0  2022-12-02  
1       60  2006.0  2022-12-02  
2       60  2006.0  2022-12-02  
3       60  2008.0  2022-12-02  
4       60  2008.0  2022-12-02  


In [3]:
import pandas as pd
import folium

# ========================
# 1. 그래프 읽기
# ========================

df = pd.read_csv("../data/서울시_태양광발전사업_정보.csv", encoding='cp949')

# ========================
# 2. 주소에서 "구" 추출
# ========================

# 예시) "서울시 중구 오장동 101-21" -> "중구" 추출
# .str.split(): 공백으로 문자열 나누기 -> ['서울시', '중구', '오장동', '101-21']
# .apply(): 각 행마다 함수 적요

def extract_gu(x):
    for word in x:
        if word.endswith('구'):
            return word
    return None

df['자치구'] = df['소재지도로명주소'].str.split().apply(extract_gu)

# ================================
# 3. 자치구별로 데이터 묶기 (집계)
# ================================

# groupby('자치구'): 같은 자치구끼리 그룹으로 묶기
#   - '설비용량(kW)': sum(합계)  -> 자치구별 총 발전 용량
#   - '태양광발전시설명': count(개수) -> 자치구별 발전소 개수

구별 = df.groupby('자치구').agg({
    '설비용량(kw)': 'sum',      # 발전량 합계
    '태양광발전시설명': 'count'  # 발전소 개수
}).reset_index()

# 칼럼 이름 바꾸기 (보기 좋게)
구별.columns = ['자치구', '총_설비용량', '발전소_수']

# ================================
# 4. 결과 출력 (발전소 많은 순서)
# ================================

print("="*35)
print("서울시 자치구별 태양광 발전소 현황")
print("="*35)
# sort_value('총_설비용량', ascending=False): 설비용량 큰 순서대로 정렬
print(구별.sort_values('총_설비용량', ascending=False).to_string(index=False))

# CSV로도 저장
# 구별.sort_values('총_설비용량', ascending=False) \
#    .to_csv("서울시_자치구별_태양광_집계.csv", 
#            index=False, 
#            encoding='utf-8-sig')

# ===========================
# 5. 자치구별 중심 좌표 설정
# ===========================

# 자치구 좌표
좌표 = {
    '강서구': [37.5509, 126.8495],
    '송파구': [37.5145, 127.1059],
    '동대문구': [37.5838, 127.0421],
    '강남구': [37.4979, 127.0630],
    '서초구': [37.4837, 127.0323],
    '강동구': [37.5501, 127.1237],
    '영등포구': [37.5264, 126.8963],
    '마포구': [37.5663, 126.9019],
    '양천구': [37.5240, 126.8558],
    '구로구': [37.4954, 126.8568],
    '관악구': [37.4653, 126.9516],
    '노원구': [37.6542, 127.0568],
    '은평구': [37.6176, 126.9292],
    '강북구': [37.6396, 127.0255],
    '중랑구': [37.5953, 127.0927],
    '광진구': [37.5384, 127.0822],
    '도봉구': [37.6688, 127.0471],
    '금천구': [37.4519, 126.9009],
    '서대문구': [37.5791, 126.9368],
    '종로구': [37.5735, 126.9792],
    '성동구': [37.5508, 127.0408],
    '동작구': [37.4972, 126.9522],
    '성북구': [37.6066, 127.0166],
    '용산구': [37.5311, 126.9640],
    '중구': [37.5636, 126.9979]
}

# ===================
# 6. 지도 생성
# ===================

# location=[위도, 경도]: 지도 중심 (서울 중심)
# zoom_start=11: 확대 정
m = folium.Map(location=[37.55, 126.98], zoom_start=11)

# ================================
# 7. 자치구별 마커 추가 (for문)
# ================================

# 발전소 최다/최소 자치구 표시
max_gu = 구별.loc[구별['발전소_수'].idxmax(), '자치구']
min_gu = 구별.loc[구별['발전소_수'].idxmin(), '자치구']

# 구별 데이터프레임의 각 행(row)마다 반복

for idx, row in 구별.iterrows():
    구 = row['자치구']    # 예) 강서구
    
    # 좌표 딕셔너리에 해당 자치구가 있는지 확인
    if 구 in 좌표:

        # 해당 자치구의 위도, 경도 가져오기 (lat=위도, lon=경도)
        lat, lon = 좌표[구]
        
        # 발전소 수에 따라 색상 결정
        if 구 == max_gu:
            color = 'blue'
        elif 구 == min_gu:
            color = 'purple'
        elif row['발전소_수'] >= 30:
            color = 'green'
        elif row['발전소_수'] >= 20:
            color = 'orange'
        else:
            color = 'red'

        # ================================
        # 마커 추가 (택시 예제와 동일)
        # ================================
        
        folium.Marker(
            [lat, lon],

            # popup: 마커 클릭하면 나오는 창
            # <b>: HTML 볼드체 (진하게)
            # <br>: 줄바꿈
            popup=f"""
            <b>{구}</b>
            <br>발전소: {row['발전소_수']}개
            <br>설비용량: {row['총_설비용량']:.1f}kW
            """,

            # tooltip: 마커에 마우스만 올려도 나오는 간단한 설명
            tooltip=f"{구}: {row['발전소_수']}개",
            icon=folium.Icon(color=color, icon='fa-solid fa-solar-panel', prefix='fa-solid')
        ).add_to(m)
m

서울시 자치구별 태양광 발전소 현황
 자치구   총_설비용량  발전소_수
 강서구 9528.890     34
 강동구 8159.625     28
 성동구 4166.380     12
 중랑구 2913.180     28
 강남구 2903.070     30
 송파구 2580.020     34
 노원구 2525.881     32
 광진구 2117.200     31
 서초구 1483.275     22
 금천구 1102.642     22
 구로구 1099.950     24
 도봉구 1046.885     25
 은평구  974.370     25
 관악구  852.270     27
영등포구  746.978     17
 성북구  720.390     25
동대문구  691.049     21
서대문구  685.305     19
 동작구  598.500     19
 양천구  587.777     14
 강북구  530.390     18
 마포구  361.510     11
  중구  241.610      8
 종로구  204.300      6
 용산구  133.585      6
